In [ ]:
from video_commands import *
main_video = "B0815_subtitle.mov"
width,height = get_media_dimensions(main_video)
print(width,height)

## check dimensions of a file

In [ ]:
width,height = get_media_dimensions("black.mov")
print(width,height)



## trim Video
outvideo = "test.mov"<br>
invideo = "B0815.mov"<br>
starttime,endtime = "0:10" , "0:18"<br>

In [ ]:
outvideo = "test_3min.mov"
invideo = "B0815_2flower.mov"
starttime,endtime = "17:10" , "20:10"
trim_video(invideo, starttime , endtime, outvideo)

## add a black still and insert in video<br>
input_video = "test_1min.mov"<br>
outfile = "test.mov"<br>
insert_black_stills(input_video=input_video,black_list=black_list,output_file=outfile,crf=18,preset="fast")



# # ADD multiple black stills and insert into video
<font color = "red">add def, insert multiple new black stills using concat. Ask me to delete original mov</font>



In [ ]:
from video_commands import *
input_video = "test_5min.mov"
start_time = 0#"17:00.5" #5
end_time = 5#"17:03.5"   #10
crf = 10
preset = "fast"
input_file=[["28年后的巨变","black1.mov"],["日本留学","black2.mov"],["回国梦","black3.mov"]]
#for i in input_file:
#    create_black_still(input_video, i[0],start_time , end_time, i[1], font_name="Arial",font_size=130,font_color="&H00FFFFFF")

video_duration = get_video_length(input_video)
sections = ["0:00-1:00", "1:00-2:00", "2:00-3:00",f"3:00-{video_duration}"]
input_file = "test_5min.mov"
outputfiles = ["video1.mov","video2.mov","video3.mov","video4.mov"]
#for section, outfile in zip(sections, outputfiles):
#     split_video(input_file, [section], [outfile])
video_list =["video1.mov","black1.mov","video2.mov","black2.mov","video3.mov","black3.mov","video4.mov"]
#combine_video(video_list,primary_video_index= ,output_file="test.mov", crf=40, preset="ultrafast")
combine_video(video_list,0,output_file="test_final3.mov", crf=18, preset="fast")

## Audio - converting mp3 to wav<br>
mp3_file = "music/darkness.mp3"

In [ ]:
mp3_file = "music/darkness.mp3"
mp3_to_wav(mp3_file)

## Audio - adjusting volume<br>
wav_file = "music/darkness.wav" <br>
volume_factor = 0.1

In [ ]:
wav_file = "music/darkness.wav"
volume_factor = 0.1
adjust_wav_volume(wav_file, volume_factor)

## split video into sections<br>
sections = ["0:00-5:00", "5:00-8:00", "9:00-12:00"]<br>
split_video("xxx.mov", sections)<br>
output:test_5min_0005-0100.mov,test_5min_0100-0200.mov,test_5min_0220-0300.mov


In [ ]:
sections = ["0:05-1:00", "1:00-2:00", "2:20-3:00"]
input_file = "test_5min.mov"
split_video(input_file, sections)

## extract voice from a video and convert to srt file
lang can be None


In [ ]:
input_video, output_srt,lang = "test_1min.mov", "test_1min.srt","zh"
voice_to_srt(input_video, output_srt, lang=lang)#en,ja,k

## srt to ass

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles("test_1min.mov", "test_1min.ass", "output.mov", "Arial")


## Translation and Combine Different Translations for proofreading
<h3>only for srt!!!</h3>

In [ ]:
inputfile,outputfile,lang = "test_1min.srt","test_1min_jp.srt","JA" #lang = target language
deepl_translate_srt(inputfile, outputfile ,lang=lang)#EN-US KO zh JA

In [ ]:
inputfile = "test_1min.srt"
outputfile = "test_1min_jp.srt"
combined_file = "combined_chi_jp.srt"  #new file, oneline chinese one line japanese
combine_srt(inputfile,outputfile, combined_file)

## Add grid to a screenshot for mosaic
add_numbered_grid(<br>
    input_file,output_file,width,height,interval=100,line_color="red",<br>
    number_color="yellow",number_size=30,number_offset=5)

In [ ]:
time_and_name = [
        ["1:10", "test1.png"],           # 1 minute 10 seconds
        ["70", "test2.jpg"],             # 70 seconds
        ["1:30.5", "test3.png"],         # 1 minute 30.5 seconds
        ["00:01:45", "test4.jpg"],       # 1 minute 45 seconds (full format)
        ["00:02:30.750", "test5.png"],   # 2 minutes 30.75 seconds
        ["4:41", "test6.jpg"]            # 4 minutes 41 seconds
        ]
extract_frames("test_5min.mov", time_and_name)

In [ ]:
for i in time_and_name:
    outscreenshot = f"{i[1].split(".")[0]+"_grid.png"}"
    add_numbered_grid(
    i[1],
    outscreenshot,
    width,
    height,
    interval=50,
    line_color="red",
    number_color="yellow",
    number_size=30,
    number_offset=5
)

# Apply Mosaic

## example: <br>
mosaic_list=[<br>
        [50, 50, 100, 50, 10, 0, 10],      <br>
        [800, 0, 200, 200, 15, "1:20.5", "3:00"]]  <br>
    # position of top-left corner<br>
    x=50,y=50,width=100,height=50<br>
    pixelization level=10<br>
    start seconds=0,end seconds=10<br>

In [ ]:
input_video,output_video = "test_5min.mov","test_5min_mosaic.mov"
mosaic_list = [[65, 50, 200, 170, 15, 0, 30],[width-260, 60, 285, 140, 15, "1:20.5", "2:30"]]
apply_mosaics(input_video, output_video, mosaic_list)

## srt to ass and burn in subtitles <br>
srt_file = "../中文字幕.srt" #input file <br>
ass_file = "中文字幕.ass" #new file <br>
input_video = "B0815_4letter.mov" <br>
output_video = "B0815_subtitle.mov" <br>
#srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30) <br>
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="18",preset="veryfast") <br>

In [ ]:
srt_file = "中文字幕.srt" #input file
ass_file = "中文字幕.ass" #new file
input_video = "B0815_4letter.mov"
output_video = "B0815_subtitle.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial",crf ="10",preset="fast")

In [ ]:
srt_file = "test_1min.srt" #input file
ass_file = "test_1min.ass" #new file
input_video = "test_1min.mov"
output_video = "test.mov"
srt_to_ass(srt_file, ass_file,width,height,fontname="宋体", fontsize=30)
burn_subtitles(input_video, ass_file, output_video, "Arial")

## End film
input "end_subtitle_txt" in the end film this way:<br>
编导演播：翰 光<br>
音乐：Chad Cannon<br>
照片提供： 小林熏<br>
         李晓斌<br>

The content of <b>end_subtitle_txt</b> is inserted into <b>end_subtitle_template</b> <br>
as <b>end_subtitle_ass</b> by <b><i>end_credits_ass</b></i>.<br>
Then <b><i>create_ending_film</b></i> is called to create <b>end_output</b> with <b>end_song</b> as background music.<br>
The dimension of  <b>end_output</b> is the same as the <b>main_video</b>. The background music has an adjustable volume<br>
and fade-in,fade-out time. <b>end_second</b> is the silent time before the end of the film and after the end of the song.
You can add a background video with an adjustable fade-out time. <br>
Remember to use <b><i>preprocess</b></i> to rescale it first.<br>
Add rescale to match the scale

In [ ]:
end_subtitle_ass= "2508credits.ass"
end_subtitle_txt= "end_subtitle.txt"
end_subtitle_template = "end_subtitle_template.ass"
end_song = "music/end_song.wav"
end_output3 = "end_vol3.mov"
end_credits_ass(end_subtitle_template, end_subtitle_txt, end_subtitle_ass,first_start="0:00:01.40", 
                 each_duration_s=6.0,line_offset_s=0.60, fallback_duration_s=4.0)
create_ending_film(end_subtitle_ass, end_song, end_output3, width, height,
                       volume=3, end_second=5, fadein=3, fadeout=10,
                       bg_video=None,video_fadeout=4)

## Reencode and combine 
Do the reencoding inside
output_file="B0815_final1.5.mp4" not ok,must be <br>
output_file="B0815_final1.mp4", no "." in between

In [ ]:
from video_commands import *
#main_video = "test_3min.mov"
#width,height = get_media_dimensions(main_video)
list_to_reencode = ["beginning_ban.mp4","end_vol3.mov"]
video_list = ["beginning_ban.mov","test_final3.mov","end_vol3.mov"]
#make_rescaled_image(width, height, "beginning_ban.mp4", "beginning_ban_rescaled.mp4")
#make_rescaled_image(width, height, "end_vol3.mov", "end_vol3_rescaled.mov4")
#a,b= reencode_to_match2(main_video, list_to_reencode)
#print(a)
#print(b)
#files = ["tmp/1_test_3min_reencoded.mov","tmp/0_beginning_ban_reencoded.mov","tmp/2_end_vol3_reencoded.mov"]
#print_media_info(files)

combine_video(video_list, primary_index=1,output_file="B0815_final_3min.mov", crf=18,preset="fast")


In [ ]:
#complicated method
time1,time2 = "13:09.8","13:14.9"
duration = parse_time(time2)-parse_time(time1)
print(duration)
overlay_video_split_and_combine(
    main_video="B0815_3flower.mov",
    overlay_video="video_folder/letter_muted.mp4",
    overlay_start="0:02",         # trim overlay from 1:01
    overlay_duration = duration,        # modify to the end
    main_insert_at= time1,    
    output_file="B0815_4letter.mov",# insert overlay at 5:00 of main
    crf=10,
    preset="fast",
    keep_intermediates=True
)



In [ ]:
make_rescaled_image(width, height, "班和保人.png", "test.png")

In [ ]:
#fade not ok
from video_commands import *
video_path="test_5min.mov"
video_duration = get_video_length(video_path)
video1_path = "war.mov"  #overlay video
video2_path = "test_5min.mov"   #main video
overlay_start_main = "4:00"#parse_time(video_duration)-10 #"13:09.8","13:14.9"
overlay_end_main = "4:25"#video_duration
extract_start_video1 = 20
output_filename = "test_war_fade2.mov"
overlay_video(
        video1_path,
        video2_path,
        overlay_start_main,
        overlay_end_main,
        extract_start_video1,
        output_filename,
        crf=40,
        preset="ultrafast",
        fadeout=0
    )


In [ ]:
from video_commands import *
video_path="test_5min.mov"
video1_path = "war_reencoded.mov"  #overlay video
video2_path = "test_5min.mov"   #main video
overlay_start_main = "4:30" #"13:09.8","13:14.9"
overlay_end_main =  "4:50"
extract_start_video1 = 20
output_filename = "test2.mov"
overlay_video(
        video1_path,
        video2_path,
        overlay_start_main,
        overlay_end_main,
        extract_start_video1,
        output_filename,
        crf=30,
        preset="ultrafast",
        fadeout=3
    )

1.automatic rescale of images and videos in the code or do reencoding in the preprocessing<br>
2.insert multiple black stills<br>
3.insert multiple images and videos<br>
4.overlay a single image, or list image<br>
overlay multiple videos<br>
5.overlay_black_still not working in github<br>
6. add preprocessing<br>
7. 更改片尾字幕resolution<br>
8. convert the following to python:<br>
ffmpeg -i B0815_1mosaic_better_quality.mov -i 黄花菜地_muted.mov -filter_complex "<br>
[0:v]trim=0:1601,setpts=PTS-STARTPTS[part1];<br>
color=c=black:size=854x480<br>
:d=21[blackpart];<br>
[1:v]trim=0:21,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=19:d=2:alpha=1[ov];<br>
[blackpart][ov]overlay=x=(main_w-overlay_w)/2:y=(main_h-overlay_h)/2:shortest=1[part2];<br>
[part1][part2]concat=n=2:v=1:a=0[vout]<br>
" -map "[vout]" -map 0:a -c:v libx264 -pix_fmt yuv420p -preset veryfast -crf 18 -c:a copy B0815_2flower.mov<br>
9. convert it to python for overlay. start overlay time is 789.8s, upto 794.9 <br>
ffmpeg -i B0815_subtitle.mov -i video_folder/letter_muted.mp4 -filter_complex \
"[1:v]setpts=PTS+789.8/TB[v1]; \
 [0:v][v1]overlay=0:0:enable='between(t,789.8,794.9)'" \
-c:a copy -c:v libx264 -crf 30 -preset ultrafast output.mov



<h3> ***continue here , burn in subtitle </h3>

In [ ]:
#check if different videos have the same coding for combination
#
#tmp/reencoded_clip.mp4 has issues
from video_commands import *
files = ["test_3min_reencoded.mov","war.mov",
         "war_reencoded.mov","war_reencoded1.mov","test_3min.mov","rescaled_clip_reencoded.mov"]




In [2]:
from video_commands import *
video_path="test_5min.mov"
video_duration = get_video_length(video_path)
video1_path = "war.mov"  #overlay video
video2_path = "test_5min.mov"   #main video
overlay_start_main = "4:30"#parse_time(video_duration)-10 #"13:09.8","13:14.9"
overlay_end_main = "4:50"#video_duration
extract_start_video1 = 20
output_filename = "test_war_fade2.mov"
overlay_video_test(
        video1_path,
        video2_path,
        overlay_start_main,
        overlay_end_main,
        extract_start_video1,
        output_filename,
        crf=23,
        preset="veryfast",
        fadeout=0
    )

temp_dir:  /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m
Extracting clip from overlay video...
FFmpeg command: ffmpeg -ss 20.0 -i war.mov -t 20.0 -c copy -y /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip.mp4
Creating video of 0 sound volume : /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip_muted.mp4
FFmpeg command: ffmpeg -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip.wav
Extracting audio: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip.wav
Successfully created:
  - 0 volume video: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip_muted.mp4
  - Audio: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/extracted_clip.wav
Rescaling the muted clip...
FFmpeg command: ffmpeg -y -i /var/folders/9g/509r85b95jz43

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

FFmpeg command: ffmpeg -y -i test_5min_tmp.mov -vf scale=854:480,pad=854:480:0:0:black -c:v libx264 -preset fast -crf 18 -c:a copy test_5min_reencoded.mov
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfi

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

✅ Final re-encoded + padded file: test_5min_reencoded_padded.mov
⚡ Re-encoding audio to PCM: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/rescaled_clip.mp4 → rescaled_clip_tmp.mov


...

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

✅ Final re-encoded + padded file: rescaled_clip_reencoded_padded.mov


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

Creating final overlay...
FFmpeg command: ffmpeg -i test_5min_reencoded_padded.mov -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpyvcbpb8m/rescaled_clip_reencoded_padded.mov -filter_complex [0:v]trim=0:270.0,setpts=PTS-STARTPTS[part1];color=c=black:size=854x480:d=20.0[black];[1:v]trim=0:20.0,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=20.0:d=0:alpha=1[ov];[black][ov]overlay=x=(main_w-overlay_w)/2:y=(main_h-overlay_h)/2:shortest=1[part2];[0:v]trim=290.0,setpts=PTS-STARTPTS[part3];[part1][part2][part3]concat=n=3:v=1:a=0[vout] -map [vout] -map 0:a -c:v libx264 -pix_fmt yuv420p -preset veryfast -crf 23 -c:a aac -b:a 192k -movflags +faststart -y test_war_fade2.mov
Final video saved as test_war_fade2.mov


[out#0/mov @ 0x7fdfdca04e40] video:20023KiB audio:7056KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.908244%
frame= 7500 fps=113 q=-1.0 Lsize=   27326KiB time=00:04:59.92 bitrate= 746.4kbits/s speed=4.54x elapsed=0:01:06.08    
[libx264 @ 0x7fdfdca05f00] frame I:67    Avg QP:18.76  size: 20344
[libx264 @ 0x7fdfdca05f00] frame P:2769  Avg QP:20.20  size:  4704
[libx264 @ 0x7fdfdca05f00] frame B:4664  Avg QP:22.47  size:  1311
[libx264 @ 0x7fdfdca05f00] consecutive B-frames:  2.6% 41.9%  4.9% 50.6%
[libx264 @ 0x7fdfdca05f00] mb I  I16..4: 18.0% 45.0% 37.0%
[libx264 @ 0x7fdfdca05f00] mb P  I16..4:  8.2% 14.1%  1.6%  P16..4: 23.9% 10.1%  4.5%  0.0%  0.0%    skip:37.6%
[libx264 @ 0x7fdfdca05f00] mb B  I16..4:  1.4%  2.8%  0.2%  B16..8: 12.2%  3.6%  0.4%  direct: 5.3%  skip:74.1%  L0:41.3% L1:48.3% BI:10.5%
[libx264 @ 0x7fdfdca05f00] 8x8 transform intra:59.2% inter:40.2%
[libx264 @ 0x7fdfdca05f00] coded y,uvDC,uvAC intra: 40.0% 34.8% 6.0% inter: 3.9% 9.5% 0.0%
[l

In [1]:
#tmp/rescaled_clip's codec , use the ffmpeg command directly
from video_commands import *
video_path="test_3min.mov"
video1_path = "war.mov"  #overlay video
video2_path = "test_3min.mov"   #main video
overlay_start_main = "1:30" #"13:09.8","13:14.9"
overlay_end_main =  "1:50"
extract_start_video1 = 2
output_filename = "war_again_fade.mov"
overlay_video_test(
        video1_path,
        video2_path,
        overlay_start_main,
        overlay_end_main,
        extract_start_video1,
        output_filename,
        crf=18,
        preset="fast",
        fadeout=5
    )

temp_dir:  /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f
Extracting clip from overlay video...
FFmpeg command: ffmpeg -ss 2.0 -i war.mov -t 20.0 -c copy -y /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip.mp4
Creating video of 0 sound volume : /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip_muted.mp4
FFmpeg command: ffmpeg -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip.wav
Extracting audio: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip.wav
Successfully created:
  - 0 volume video: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip_muted.mp4
  - Audio: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/extracted_clip.wav
Rescaling the muted clip...
FFmpeg command: ffmpeg -y -i /var/folders/9g/509r85b95jz438

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

FFmpeg command: ffmpeg -y -i test_3min_tmp.mov -vf scale=854:480,pad=854:480:0:0:black -c:v libx264 -preset fast -crf 18 -c:a copy test_3min_reencoded.mov
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfi

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

✅ Final re-encoded + padded file: test_3min_reencoded_padded.mov
⚡ Re-encoding audio to PCM: /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/rescaled_clip.mp4 → rescaled_clip_tmp.mov


...

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

✅ Final re-encoded + padded file: rescaled_clip_reencoded_padded.mov


ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/8.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enabl

Creating final overlay...
FFmpeg command: ffmpeg -i test_3min_reencoded_padded.mov -i /var/folders/9g/509r85b95jz43885pc6n7zlr0000gn/T/tmpp8frhk_f/rescaled_clip_reencoded_padded.mov -filter_complex [0:v]trim=0:90.0,setpts=PTS-STARTPTS[part1];color=c=black:size=854x480:d=20.0[black];[1:v]trim=0:20.0,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=15.0:d=5:alpha=1[ov];[black][ov]overlay=x=(main_w-overlay_w)/2:y=(main_h-overlay_h)/2:shortest=1[part2];[0:v]trim=110.0,setpts=PTS-STARTPTS[part3];[part1][part2][part3]concat=n=3:v=1:a=0[vout] -map [vout] -map 0:a -c:v libx264 -pix_fmt yuv420p -preset fast -crf 18 -c:a aac -b:a 192k -movflags +faststart -y war_again_fade.mov
Final video saved as war_again_fade.mov


[mov @ 0x7fa9a1707e00] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mov @ 0x7fa9a1707d40] video:15668KiB audio:4144KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.749828%
frame= 4500 fps= 53 q=-1.0 Lsize=   19961KiB time=00:02:59.92 bitrate= 908.9kbits/s speed=2.12x elapsed=0:01:24.88    
[libx264 @ 0x7fa9a1709b00] frame I:33    Avg QP:11.62  size: 39837
[libx264 @ 0x7fa9a1709b00] frame P:1564  Avg QP:13.76  size:  5953
[libx264 @ 0x7fa9a1709b00] frame B:2903  Avg QP:15.97  size:  1866
[libx264 @ 0x7fa9a1709b00] consecutive B-frames:  1.7% 36.0%  2.7% 59.6%
[libx264 @ 0x7fa9a1709b00] mb I  I16..4: 22.2% 52.2% 25.6%
[libx264 @ 0x7fa9a1709b00] mb P  I16..4:  7.4% 11.1%  1.1%  P16..4: 31.6% 11.7%  4.3%  0.0%  0.0%    skip:32.7%
[libx264 @ 0x7fa9a1709b00] mb B  I16..4:  1.5%  2.2%  0.2%  B16..8: 18.7%  4.9%  0.3%  direct:10.7%  skip:61.5%  L0:43.8% L1:49.5% BI: 6.7%
[libx264 @ 0x7fa9a1709b00] 8x8 transform intra:55.9% inter:60.9

overlay video command <br>
ffmpeg -i test_5min.mov -i war_reencoded.mov -filter_complex " <br>
              [0:v]trim=0:270,setpts=PTS-STARTPTS[part1]; <br>
              color=c=black:size=854x480:d=25[black]; <br>
              [1:v]trim=20:40,setpts=PTS-STARTPTS,format=rgba,fade=t=out:st=18:d=2:alpha=1[ov]; <br>
              [black][ov]overlay=x=(main_w-overlay_w)/2:y=(main_h-overlay_h)/2:shortest=1[part2]; <br>
              [0:v]trim=290,setpts=PTS-STARTPTS[part3]; <br>
              [part1][part2][part3]concat=n=3:v=1:a=0[vout] <br>
              " -map "[vout]" -map 0:a -c:v libx264 -pix_fmt yuv420p -preset veryfast -crf 30 -c:a copy -y test2.mov <br> <br>
              
             ffmpeg -i {video2_path} -i {temp_rescaled} -filter_complex " <br>
              [0:v]trim=0:{start_time_main},setpts=PTS-STARTPTS[part1]; <br>
              color=c=black:size={main_width}x{main_height}:d={duration}[black]; <br>
              [1:v]trim={extract_start}:{extract_start+duration},setpts=PTS-STARTPTS,format=rgba,fade=t=out:st={duration-fadeout}:d={fadeout}:alpha=1[ov]; <br>
              [black][ov]overlay=x=(main_w-overlay_w)/2:y=(main_h-overlay_h)/2:shortest=1[part2]; <br>
              [0:v]trim={end_time_main},setpts=PTS-STARTPTS[part3]; <br>
              [part1][part2][part3]concat=n=3:v=1:a=0[vout] <br>
              " -map "[vout]" -map 0:a -c:v libx264 -pix_fmt yuv420p -preset {preset} -crf {crf} -c:a copy -y {output_filename} <br>
            

In [ ]:
from video_commands import*
primary_video = "test_5min.mov"
list_to_reencode = ["war.mov"]
reencode_to_match(primary_video, list_to_reencode)

In [2]:
from video_commands import * 
success, temp_muted, audio_file = separate_audio_video("war_muted.mov",".",0)
main_video = "test_3min.mov"
list_to_reencode = ['war_muted.mov']
#a,b = reencode_to_match(main_video, list_to_reencode,crf="18",preset="fast")
#print(a)
a = ["tmp/test_3min_reencoded_padded.mov","tmp/rescaled_clip_reencoded_padded.mov"]
print_media_info(a)



Creating video of 0 sound volume : ./war_muted_muted.mov
FFmpeg command: ffmpeg -i war_muted.mov -vn -acodec pcm_s16le -ar 44100 -ac 2 ./war_muted.wav
Extracting audio: ./war_muted.wav
Successfully created:
  - 0 volume video: ./war_muted_muted.mov
  - Audio: ./war_muted.wav
---- tmp/test_3min_reencoded_padded.mov ----
Video: h264,854,480,yuv420p,25/1
Audio: aac,44100,2
video start and end time:0.000000,180.000000
audio start and end time:0.000000,180.000000

---- tmp/rescaled_clip_reencoded_padded.mov ----
Video: h264,854,480,yuv420p,25/1
Audio: aac,44100,2
video start and end time:0.000000,20.160000
audio start and end time:0.000000,20.160000



In [2]:
from video_commands import *
a = get_video_length("test_3min.mov")
print(a)
a = get_video_length("tmp/rescaled_clip_reencoded_padded.mov")
print(a)

03:00.015
0:20.160


## notes
reencode_to_match()<br>
reencode videos in aac, but supposed to be in pcm